# Overfitting - Regularization in Regression

### $$J(W) = Cost(X, y, W) + \lambda R(W)$$
$$J(W) = error(\hat{y}, y) + \lambda ||W||_i $$

In [1]:
# %matplotlib inline #Jupyter Notebook
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score

import warnings

# plotting setup

In [ ]:
# plt.figure(figsize=(10,5), dpi=150)
plt.rcParams['figure.figsize'] = [10,5]
plt.rcParams['figure.dpi'] = 150
plt.style.use('Solarize_Light2')
np.random.seed(42)
np.set_printoptions(precision=2, linewidth=50)

## Regression with Regularization